In [1]:
import pandas as pd
import numpy as np

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
!pip install umap-learn[plot]
!pip install fastparquet
import umap
import umap.plot

     |████████████████████████████████| 88 kB 4.2 MB/s 
     |████████████████████████████████| 1.1 MB 14.2 MB/s 
     |████████████████████████████████| 15.8 MB 34.7 MB/s 
     |████████████████████████████████| 76 kB 5.0 MB/s 
     |████████████████████████████████| 136 kB 48.7 MB/s 
     |████████████████████████████████| 837 kB 50.3 MB/s 
     |████████████████████████████████| 830 kB 49.1 MB/s 
     |████████████████████████████████| 822 kB 48.8 MB/s 
     |████████████████████████████████| 802 kB 50.8 MB/s 
     |████████████████████████████████| 802 kB 50.5 MB/s 
     |████████████████████████████████| 793 kB 48.6 MB/s 
     |████████████████████████████████| 793 kB 41.6 MB/s 
     |████████████████████████████████| 791 kB 49.3 MB/s 
     |████████████████████████████████| 786 kB 47.5 MB/s 
     |████████████████████████████████| 779 kB 50.7 MB/s 
     |████████████████████████████████| 778 kB 43.6 MB/s 
     |████████████████████████████████| 776 kB 49.9 MB/s 
     |███████████

     |████████████████████████████████| 1.5 MB 8.2 MB/s 
     |████████████████████████████████| 1.6 MB 55.1 MB/s 


In [4]:
!unzip -o /content/drive/MyDrive/CS4220_mp2/CS4220/datasets/validation-20220402T110325Z-001.zip
! unzip -o /content/drive/MyDrive/CS4220_mp2/CS4220/datasets/val_11_16_data.zip -d validation/
! unzip -o /content/drive/MyDrive/CS4220_mp2/CS4220/datasets/val_11_16_labels.zip -d validation/

Archive:  /content/drive/MyDrive/CS4220_mp2/CS4220/datasets/validation-20220402T110325Z-001.zip
  inflating: validation/val7_labels.txt  
  inflating: validation/val8_labels.txt  
  inflating: validation/val10_labels.txt  
  inflating: validation/val9_labels.txt  
  inflating: validation/val6_labels.txt  
  inflating: validation/val3_labels.txt  
  inflating: validation/val5_abundance.txt  
  inflating: validation/val3_abundance.txt  
  inflating: validation/val4_labels.txt  
  inflating: validation/val1_labels.txt  
  inflating: validation/val9_abundance.txt  
  inflating: validation/val10_abundance.txt  
  inflating: validation/val6_abundance.txt  
  inflating: validation/val1_abundance.txt  
  inflating: validation/val5_labels.txt  
  inflating: validation/val7_abundance.txt  
  inflating: validation/val2_labels.txt  
  inflating: validation/val4_abundance.txt  
  inflating: validation/val2_abundance.txt  
  inflating: validation/val8_abundance.txt  
  inflating: validation/genome_r

In [5]:
dataset_folder = "./drive/MyDrive/CS4220_mp2/CS4220/datasets/"
labels_df = pd.read_csv(dataset_folder + "labels.csv")
print(labels_df.shape)

(860663, 4)


In [44]:
labels_df.head()

,Unnamed: 0,genome_name,rl,rq
0,0,staphylococcus_aureus,962,98.26
1,0,staphylococcus_aureus,962,98.26
2,1,staphylococcus_aureus,704,90.25
3,2,staphylococcus_aureus,2617,96.20
4,3,staphylococcus_aureus,4169,99.38


In [6]:
df_train = pd.read_csv(dataset_folder + "perfect_reads_6mer.gz", sep="\t", header=None, nrows=100000)
print(df_train.shape)
df_train.head()

(100000, 2081)


,0,1,2,3,4,5,6,7,8,9,...,2071,2072,2073,2074,2075,2076,2077,2078,2079,2080
0,0.003247,0.002165,0.002165,0.000000,0.004329,0.002165,0.001082,0.000000,0.001082,0.001082,...,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,0.006237,0.004158,0.001040,0.002079,0.003119,0.003119,0.004158,0.001040,0.004158,0.000000,...,0.002079,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,0.004261,0.002841,0.001420,0.004261,0.005682,0.007102,0.005682,0.002841,0.001420,0.002841,...,0.000000,0.00142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,0.000000,0.001528,0.002293,0.001146,0.001911,0.001911,0.000764,0.001528,0.003439,0.001911,...,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,0.001919,0.003838,0.001679,0.002399,0.003838,0.002878,0.001919,0.002399,0.002399,0.001199,...,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [7]:
df_train.drop(2080, axis = 1, inplace=True)

In [8]:
df_train["labels"] = labels_df["genome_name"].iloc[0:100000]

In [9]:
# sample_size = 2078
# df_train = df_train.groupby('labels').apply(lambda x: x.sample(sample_size))

In [10]:
def get_genus(x):
  genus = x.split("_")[0]
  # species = x.split("_")[1]
  return genus

In [11]:
def get_species(x):
  if x == "decoy":
    return "decoy"
  species = x.split("_")[1]
  return species

In [12]:
df_train["genus"] = df_train["labels"].apply(get_genus)
df_train["species"] = df_train["labels"].apply(get_species)

In [13]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,2073,2074,2075,2076,2077,2078,2079,labels,genus,species
0,0.003247,0.002165,0.002165,0.000000,0.004329,0.002165,0.001082,0.000000,0.001082,0.001082,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,staphylococcus_aureus,staphylococcus,aureus
1,0.006237,0.004158,0.001040,0.002079,0.003119,0.003119,0.004158,0.001040,0.004158,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,staphylococcus_aureus,staphylococcus,aureus
2,0.004261,0.002841,0.001420,0.004261,0.005682,0.007102,0.005682,0.002841,0.001420,0.002841,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,staphylococcus_aureus,staphylococcus,aureus
3,0.000000,0.001528,0.002293,0.001146,0.001911,0.001911,0.000764,0.001528,0.003439,0.001911,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,staphylococcus_aureus,staphylococcus,aureus
4,0.001919,0.003838,0.001679,0.002399,0.003838,0.002878,0.001919,0.002399,0.002399,0.001199,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,staphylococcus_aureus,staphylococcus,aureus


In [14]:
from sklearn.preprocessing import LabelEncoder
le_genus = LabelEncoder()
le_species = LabelEncoder()
le_labels = LabelEncoder()

In [15]:
df_train["genus_encoded"] = le_genus.fit_transform(df_train["genus"])
df_train["species_encoded"] = le_species.fit_transform(df_train["species"])
df_train["labels_encoded"] = le_labels.fit_transform(df_train["labels"])

In [16]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,2076,2077,2078,2079,labels,genus,species,genus_encoded,species_encoded,labels_encoded
0,0.003247,0.002165,0.002165,0.000000,0.004329,0.002165,0.001082,0.000000,0.001082,0.001082,...,0.0,0.0,0.0,0.0,staphylococcus_aureus,staphylococcus,aureus,6,1,8
1,0.006237,0.004158,0.001040,0.002079,0.003119,0.003119,0.004158,0.001040,0.004158,0.000000,...,0.0,0.0,0.0,0.0,staphylococcus_aureus,staphylococcus,aureus,6,1,8
2,0.004261,0.002841,0.001420,0.004261,0.005682,0.007102,0.005682,0.002841,0.001420,0.002841,...,0.0,0.0,0.0,0.0,staphylococcus_aureus,staphylococcus,aureus,6,1,8
3,0.000000,0.001528,0.002293,0.001146,0.001911,0.001911,0.000764,0.001528,0.003439,0.001911,...,0.0,0.0,0.0,0.0,staphylococcus_aureus,staphylococcus,aureus,6,1,8
4,0.001919,0.003838,0.001679,0.002399,0.003838,0.002878,0.001919,0.002399,0.002399,0.001199,...,0.0,0.0,0.0,0.0,staphylococcus_aureus,staphylococcus,aureus,6,1,8


In [17]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

In [18]:
def precision_per_patient(patient_id, preds):
    df_true = pd.read_csv('./validation/val{}_labels.txt'.format(patient_id))
    tp, fp=0,0
    print('my predition(s) for patient {}:'.format(patient_id))
    print(preds)
    print('true pathogen')
    print(df_true['true_label'].values)
    for item in np.unique(preds):
        if item in df_true['true_label'].values:
            tp+=1
        else:
            fp+=1
    return tp/(tp+fp)

In [19]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=42)

In [20]:
df_train["genus_encoded"].value_counts()

2    55249
3    12051
0     8558
5     7568
1     5847
6     5490
4     2637
7     2600
Name: genus_encoded, dtype: int64

In [21]:
clf.fit(df_train.iloc[:,0:2080], df_train["genus_encoded"].values)

RandomForestClassifier(random_state=42)

In [22]:
le_labels.classes_

array(['burkholderia_pseudomallei', 'corynebacterium_diphtheriae',
       'corynebacterium_ulcerans', 'decoy', 'mycobacterium_tuberculosis',
       'mycobacterium_ulcerans', 'neisseria_gonorrhoeae',
       'pseudomonas_aeruginosa', 'staphylococcus_aureus',
       'staphylococcus_pyogenes', 'streptococcus_pneumoniae'],
      dtype=object)

In [23]:
le_genus.classes_

array(['burkholderia', 'corynebacterium', 'decoy', 'mycobacterium',
       'neisseria', 'pseudomonas', 'staphylococcus', 'streptococcus'],
      dtype=object)

In [24]:
import pickle
# pickle.dump( clf, open( "./drive/MyDrive/CS4220_mp2/rf_100000_genus.pkl", "wb" ) )

In [25]:
cornyebacterium_df = df_train[df_train.genus == "corynebacterium"]
clf_corynebacterium = RandomForestClassifier(random_state = 3)
print(cornyebacterium_df.shape)
clf_corynebacterium.fit(cornyebacterium_df.iloc[:,0:2080], cornyebacterium_df["labels_encoded"].values)

(5847, 2086)


RandomForestClassifier(random_state=3)

In [26]:
mycobacterium_df = df_train[df_train.genus == "mycobacterium"]
clf_mycobacterium = RandomForestClassifier(random_state = 3)
print(mycobacterium_df.shape)
clf_mycobacterium.fit(mycobacterium_df.iloc[:,0:2080], mycobacterium_df["labels_encoded"].values)

(12051, 2086)


RandomForestClassifier(random_state=3)

In [27]:
staphylococcus_df = df_train[df_train.genus == "staphylococcus"]
clf_staphylococcus = RandomForestClassifier(random_state = 3)
print(staphylococcus_df.shape)
clf_staphylococcus.fit(staphylococcus_df.iloc[:,0:2080], staphylococcus_df["labels_encoded"].values)

(5490, 2086)


RandomForestClassifier(random_state=3)

In [38]:
counts = list(yfinal["res"].value_counts())
mypreds = list(le_labels.inverse_transform(np.array(yfinal["res"].value_counts().index)))
filterdf = pd.DataFrame(columns = ["counts", "genome"])
filterdf["counts"] = counts
filterdf["genome"] = mypreds
filterdf

,counts,genome
0,9936,decoy
1,48,neisseria_gonorrhoeae
2,4,mycobacterium_ulcerans
3,3,mycobacterium_tuberculosis
4,3,burkholderia_pseudomallei
5,1,corynebacterium_diphtheriae


In [39]:
filterdf.iloc[0:2]

,counts,genome
0,9936,decoy
1,48,neisseria_gonorrhoeae


In [41]:
# fig, ax = plt.subplots(16, 1, figsize=(20,20))
all_precision = []
for patient_id in range(1,17):
    # kmeans = KMeans(random_state=42, n_clusters = 2)
    print('predicting for patient {}'.format(patient_id))
    df_test = pd.read_csv('./validation/val{}_6mer.gz'.format(patient_id), header=None, sep='\t')
    # mapped = umap.UMAP().fit_transform(df_test.iloc[:,:-1])
    # df_test["clusters"] = gmm.fit_predict(df_test.iloc[:,:-1])
    # sns.scatterplot(x=mapped[:,0], y=mapped[:, 1], ax=ax[patient_id-1])
    # ax[patient_id] = umap.plot.points(mapped, width=100, height=100)
    # min_cluster_idx = df_test["clusters"].value_counts().sort_values().index[0]
    # subset = df_test[df_test["clusters"] == min_cluster_idx]
    subset = df_test
    ypred = clf.predict(subset.iloc[:, 0:2080])
    # ypred = clf_rf.predict(subset.iloc[:, important_features_idx])
    pred_df = pd.DataFrame(ypred, columns = ["pred_class"])
    # print(pred_df["pred_class"].value_counts())
    # mycobacterium, cornyebacterium or staphylococcus - do species classification
    corny_idx = list(pred_df[pred_df.pred_class == 1].index)
    myc_idx = list(pred_df[pred_df.pred_class == 3].index)
    staph_idx = list(pred_df[pred_df.pred_class == 6].index)
    # genus
    # ['burkholderia', 'corynebacterium', 'decoy', 'mycobacterium',
    #   'neisseria', 'pseudomonas', 'staphylococcus', 'streptococcus']
    # species
    # ['burkholderia_pseudomallei', 'corynebacterium_diphtheriae',
    #    'corynebacterium_ulcerans', 'decoy', 'mycobacterium_tuberculosis',
    #    'mycobacterium_ulcerans', 'neisseria_gonorrhoeae',
    #    'pseudomonas_aeruginosa', 'staphylococcus_aureus',
    #    'staphylococcus_pyogenes', 'streptococcus_pneumoniae']
    preds_final = []
    for idx, row in pred_df.iterrows():
      if idx not in corny_idx + myc_idx + staph_idx:
        if row["pred_class"] == 0: # genus = burkholderia
          preds_final.append(0)
        elif row["pred_class"] == 2: # genus = decoy
          preds_final.append(3)
        elif row["pred_class"] == 4: # genus = neisseria
          preds_final.append(6)
        elif row["pred_class"] == 5: # genus = pseudomonas
          preds_final.append(7)
        elif row["pred_class"] == 7: # genus = streptococcus
          preds_final.append(10)
      else:
        if idx in corny_idx:
          preds_final.append(clf_corynebacterium.predict(subset.iloc[idx, 0:2080].values.reshape(1,-1))[0])
        elif idx in myc_idx:
          preds_final.append(clf_mycobacterium.predict(subset.iloc[idx,0:2080].values.reshape(1,-1))[0])
        elif idx in staph_idx:
          preds_final.append(clf_staphylococcus.predict(subset.iloc[idx,0:2080].values.reshape(1,-1))[0])
    final_predictions = le_labels.inverse_transform(np.unique(preds_final))
    # print(le_genus.inverse_transform(np.array(pred_df["pred_class"].value_counts().index)))
    yfinal = pd.DataFrame(preds_final, columns=["res"])
    print(yfinal["res"].value_counts())
    counts = list(yfinal["res"].value_counts())
    mypreds = list(le_labels.inverse_transform(np.array(yfinal["res"].value_counts().index)))
    print(mypreds)
    filterdf = pd.DataFrame(columns = ["counts", "genome"])
    filterdf["counts"] = counts
    filterdf["genome"] = mypreds
    final_predictions = list(filterdf.iloc[0:2]["genome"])
    final_predictions = [x for x in final_predictions if x != "decoy"]
    # final_predictions = le_genus.inverse_transform(np.array(pred_df["pred_class"].value_counts().index))
    precision = precision_per_patient(patient_id, final_predictions)
    print('precision: {}'.format(precision))
    all_precision.append(precision)

predicting for patient 1
3    19814
8      179
1        2
4        1
2        1
Name: res, dtype: int64
['decoy', 'staphylococcus_aureus', 'corynebacterium_diphtheriae', 'mycobacterium_tuberculosis', 'corynebacterium_ulcerans']
my predition(s) for patient 1:
['staphylococcus_aureus']
true pathogen
['staphylococcus_aureus']
precision: 1.0
predicting for patient 2
3    19956
9       37
1        2
4        1
2        1
Name: res, dtype: int64
['decoy', 'staphylococcus_pyogenes', 'corynebacterium_diphtheriae', 'mycobacterium_tuberculosis', 'corynebacterium_ulcerans']
my predition(s) for patient 2:
['staphylococcus_pyogenes']
true pathogen
['staphylococcus_pyogenes']
precision: 1.0
predicting for patient 3
3    19859
2       71
0       61
1        3
4        2
5        1
Name: res, dtype: int64
['decoy', 'corynebacterium_ulcerans', 'burkholderia_pseudomallei', 'corynebacterium_diphtheriae', 'mycobacterium_tuberculosis', 'mycobacterium_ulcerans']
my predition(s) for patient 3:
['corynebacter

In [42]:
print([ 'patient {}: {}'.format(c,item) for c, item in enumerate(all_precision, start=1)], 'avg: {}'.format(np.mean(all_precision)))

['patient 1: 1.0', 'patient 2: 1.0', 'patient 3: 1.0', 'patient 4: 0.0', 'patient 5: 1.0', 'patient 6: 1.0', 'patient 7: 1.0', 'patient 8: 1.0', 'patient 9: 0.0', 'patient 10: 1.0', 'patient 11: 0.0', 'patient 12: 1.0', 'patient 13: 1.0', 'patient 14: 1.0', 'patient 15: 1.0', 'patient 16: 0.0'] avg: 0.75


In [43]:
pickle.dump( clf, open( "./drive/MyDrive/CS4220_mp2/rf_100000_genus.pkl", "wb" ) )
pickle.dump(clf_corynebacterium, open("./drive/MyDrive/CS4220_mp2/clf_cornybacterium.pkl", "wb"))
pickle.dump(clf_mycobacterium, open("./drive/MyDrive/CS4220_mp2/clf_mycobacterium.pkl", "wb"))
pickle.dump(clf_staphylococcus, open("./drive/MyDrive/CS4220_mp2/clf_staphylococcus.pkl", "wb"))